In [1]:
# Import packages
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from tqdm import tqdm

In [2]:
# Model architecture
class Net(nn.Module):
    # Structure of the NN
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)  # Input? Output? RF
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.drop1 = nn.Dropout2d(p=0.1)

        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.drop2 = nn.Dropout2d(p=0.1)

        self.conv5 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, padding=0)
        self.bn5 = nn.BatchNorm2d(num_features=16)
        self.conv6 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(num_features=16)

        self.gap1 = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc1 = nn.Linear(in_features=16, out_features=10)

    # Feed forward of the NN
    def forward(self, x):
        x = self.drop1(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
        x = self.drop2(self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))))
        x = self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.gap1(x)
        x = x.view(-1, 16)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

In [3]:
# Model summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
         Dropout2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9           [-1, 32, 14, 14]           9,248
      BatchNorm2d-10           [-1, 32, 14, 14]              64
        MaxPool2d-11             [-1, 32, 7, 7]               0
        Dropout2d-12             [-1, 32, 7, 7]               0
           Conv2d-13             [-1, 16, 7, 7]             528
      BatchNorm2d-14             [-1, 1

In [4]:
# Data transformation
train_transform = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation((-15., 15.), fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

# Fetch MNIST data
train_data = datasets.MNIST('../data', train=True, download=True, transform=train_transform)
test_data = datasets.MNIST('../data', train=False, download=True, transform=test_transform)


# Data loader
torch.manual_seed(10)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.55MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.09MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.68MB/s]


In [5]:
# Correct predictions function
def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

# Train workflow
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)  # Prediction
        loss = F.nll_loss(output, target)  # Loss calculation
        loss.backward()  # Backpropagation
        optimizer.step()
        correct += GetCorrectPredCount(output, target)
        processed += len(data)
        pbar.set_description(desc= f'Train: Loss={loss.item():.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:.2f}')

# Test workflow
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            # pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # correct += pred.eq(target.view_as(pred)).sum().item()
            correct += GetCorrectPredCount(output, target)

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# Model training and results
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(1, 20):
    print(f'Epoch {epoch}, LR: {scheduler.get_last_lr()[0]}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Epoch 1, LR: 0.01


Train: Loss=0.2070 Batch_id=468 Accuracy=77.69: 100%|██████████| 469/469 [00:24<00:00, 19.37it/s]


Test set: Average loss: 0.1266, Accuracy: 9651/10000 (96.51%)

Epoch 2, LR: 0.01


Train: Loss=0.1722 Batch_id=468 Accuracy=95.76: 100%|██████████| 469/469 [00:23<00:00, 19.79it/s]


Test set: Average loss: 0.0589, Accuracy: 9829/10000 (98.29%)

Epoch 3, LR: 0.01


...

Test set: Average loss: 0.0177, Accuracy: 9948/10000 (99.48%)

Epoch 18, LR: 0.001


Train: Loss=0.0206 Batch_id=468 Accuracy=98.83: 100%|██████████| 469/469 [00:23<00:00, 20.04it/s]


Test set: Average loss: 0.0183, Accuracy: 9946/10000 (99.46%)

Epoch 19, LR: 0.001


Train: Loss=0.0698 Batch_id=468 Accuracy=98.82: 100%|██████████| 469/469 [00:23<00:00, 19.69it/s]


Test set: Average loss: 0.0180, Accuracy: 9944/10000 (99.44%)

